In [2]:
import numpy as np
import pandas as pd
import sys
sys.path.append(r'/Users/zhouzhan/Documents/codes/python_code/liver_disease/liver_disease/')
# sys.path.append(r'E:/liver_disease/liver_disease')
import constants
from sklearn.model_selection import train_test_split

In [39]:
'''
汇总表-未做特征选择:    MERGE_CSV_PATH
汇总表-特征选择后:     SELECTION_MERGE_CSV_PATH
人工选择特征:         SYMP_MAIN_ACC_DIAGNOSIS_PATH
SMOTE:              ANALYSIS_SMOTE_MERGE_CSV_PATH
SMOTE Borderline1:  ANALYSIS_SMOTE_BORDERLINE1_MERGE_CSV_PATH
SMOTE_D:            ANALYSIS_SMOTE_D_MERGE_CSV_PATH
SMOTE_BORDERLINE_D: ANALYSIS_SMOTE_Borderline_D_CSV_PATH
随机过采样:           ANALYSIS_RANDOM_OVER_SAMPLER_CSV_PATH
'''
PATH = constants.ANALYSIS_RANDOM_OVER_SAMPLER_CSV_PATH
print(PATH)

/Users/zhouzhan/Documents/codes/python_code/liver_disease/liver_disease/output/SMOTE/论文1.0/随机过采样-汇总表.csv


In [40]:
df = pd.read_csv(PATH)

cols = df.columns.values.tolist()
# cols.remove('INHOSPTIAL_ID')
# cols.remove('ZHENGHOU1')

X = df[cols] 
y = df['ZHENGHOU1']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state=0)

print(df.head())

   腹胀  发热  尿黄  纳差  双下肢水肿  乏力  黑便  头晕  腹痛  身目黄染  ...  嗜睡  咽部隐痛不适  干呕  排便不爽  \
0   1   0   1   0      0   0   0   0   0     1  ...   0       0   0     0   
1   0   1   1   0      0   0   0   0   0     1  ...   0       0   0     0   
2   0   0   0   0      0   0   0   0   0     0  ...   0       0   0     0   
3   1   0   1   0      0   0   0   0   0     1  ...   0       0   0     0   
4   0   1   0   0      0   0   0   0   0     0  ...   0       0   0     0   

   口淡无味  大便稀少  大便未解  肝区疼痛  大便稀溏  ZHENGHOU1  
0     0     0     0     0     0          1  
1     0     0     0     0     0          1  
2     0     0     0     0     0          1  
3     0     0     0     0     0          1  
4     0     0     0     0     0          1  

[5 rows x 58 columns]


In [41]:
# 查看诊断结果的分布情况，看是否存在数据不均衡
from collections import Counter
print('证的分布情况:',Counter(df['ZHENGHOU1']))

证的分布情况: Counter({1: 214, 2: 214})


# 决策树调参

In [42]:
# 调参-网格搜索-决策树
import sklearn.tree as tree
from sklearn.model_selection import GridSearchCV

param_grid = {'criterion':['entropy','gini'],
             'max_depth':[2,3,4,8,10],
             'min_samples_split':[2,4,6,8]}

In [43]:
clf = tree.DecisionTreeClassifier() 
clfcv = GridSearchCV(estimator=clf,param_grid=param_grid,cv=4) # ,scoring='roc_auc' 
clfcv.fit(X_train,y_train)

GridSearchCV(cv=4, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['entropy', 'gini'],
                         'max_depth': [2, 3, 4, 8, 10],
                         'min_samples_split': [2, 4, 6, 8]})

In [44]:
test_result = clfcv.predict(X_test)
# 模型评估
import sklearn.metrics as metrics
print('决策树准确度：')
print(metrics.classification_report(y_test,test_result))

决策树准确度：
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        40
           2       1.00      1.00      1.00        46

    accuracy                           1.00        86
   macro avg       1.00      1.00      1.00        86
weighted avg       1.00      1.00      1.00        86



In [45]:
# 网格搜索最优参数
print(clfcv.best_params_)

{'criterion': 'entropy', 'max_depth': 2, 'min_samples_split': 2}


# 随机森林调参

In [14]:
import sklearn.ensemble as ensemble

param_grid2 = {
    'criterion':['entropy','gini'],
    'max_depth':[5,6,7,8],
    'n_estimators':[15,20,30,40],
    'max_features':[0.4,0.5,0.6,0.8],
    'min_samples_split':[6,8,10,20],
}



In [15]:
rfc = ensemble.RandomForestClassifier()
rfc_cv = GridSearchCV(estimator=rfc, param_grid=param_grid2,cv=4)
rfc_cv.fit(X_train,y_train)


GridSearchCV(cv=4, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['entropy', 'gini'],
                         'max_depth': [5, 6, 7, 8],
                         'max_features': [0.4, 0.5, 0.6, 0.8],
                         'min_samples_split': [6, 8, 10, 20],
                         'n_estimators': [15, 20, 30, 40]})

In [16]:
predict_test = rfc_cv.predict(X_test)

print('随机森林精确度：')
print(metrics.classification_report(predict_test, y_test))

随机森林精确度：
              precision    recall  f1-score   support

           1       0.70      0.61      0.65        46
           2       0.61      0.70      0.65        40

    accuracy                           0.65        86
   macro avg       0.65      0.65      0.65        86
weighted avg       0.66      0.65      0.65        86



In [17]:
# 查看最佳参数
rfc_cv.best_params_

{'criterion': 'entropy',
 'max_depth': 7,
 'max_features': 0.6,
 'min_samples_split': 8,
 'n_estimators': 20}